## This laboratory will be conducted over two sessions
## You are expected to finish the software and test the code in simulation between the sessions! ##

# Manipulating objects

For this laboratory, we will use two robots distant from each other of 25cm as depicted in the figure below

<img src="./2_robot_config.png" width="500">

There is 1cm between the end-effector "shell" and the position of the frame as depicted in the figure below

<img src="./foot_close.png" width="300">

<img src="./box.jpg" width="200">

The objects to grasp are soft cubes with a 5cm side.
Three cubes are stacked on top of each other. The goal of the laboratory is to design a controller to will move the two cubes on the top away
and pick the bottom cube and lift it up by 10cm.

## Functions From Previous Labs

In [1]:
#setup nice plotting (use widget instead of notebook in the command below if you use jupyter lab)

# we import useful libraries
import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
from time import sleep

# this function returns the body Jacobian
from robot_utils import getBodyJacobianOrientedLikeSpatialFrame

# we import the robot class
use_real_robot = False

if use_real_robot:
    from NYUFinger.real import NYUDualFingerHardware as Robot   
else:
    from NYUFinger.sim import NYUDualFingerSimulator as Robot


# here we define the global variables for the robot size
l0 = 0.3
l1 = 0.16
l2 = 0.16
l3 = 0.014

def translate(vector):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[0][3] = vector[0]
    transform[1][3] = vector[1]
    transform[2][3] = vector[2]

    ### we return the object
    return transform

def rotateX(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[1][1] = np.cos(angle)
    transform[1][2] = -(np.sin(angle))
    transform[2][1] = np.sin(angle)
    transform[2][2] = np.cos(angle)

    ### we return the object
    return transform

def rotateY(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[0][0] = np.cos(angle)
    transform[0][2] = (np.sin(angle))
    transform[2][0] = -np.sin(angle)
    transform[2][2] = np.cos(angle)
    
    ### we return the object
    return transform

def rotateZ(angle):
    # we allocate a 4x4 array (as identity since this corresponds to no motion)
    transform = np.eye(4)
    # here you can fill the rest of the transform
    ##WRITE CODE HERE
    transform[0][0] = np.cos(angle)
    transform[0][1] = -(np.sin(angle))
    transform[1][0] = np.sin(angle)
    transform[1][1] = np.cos(angle)
    
    ### we return the object
    return transform
def forward_kinematics(theta):
    # assume theta is a vector, theta[0] = theta0, theta[1] = theta1, etc.
    # we define the lengths of the robot
    l0 = 0.3
    l1 = 0.16
    l2 = 0.16
    l3 = 0.014
    
    ###WRITE YOUR CODE HERE
    t_SH1 = translate([l0,0,0]) @ rotateX(theta[0]) # translation in x by l0 and rotation in x by theta[0]

    t_H1H2 = translate([0,0,l3]) @ rotateZ(theta[1])#translation in z by l3 and rotation in z by theta[1]

    t_H2K = translate([0,-l1,0]) @ rotateZ(theta[2]) #translation in y by l1 and rotation in z by theta[2]

    t_KF = translate([0,-l2,0]) #translation in y by l2
    
    t_SF = t_SH1 @ t_H1H2 @ t_H2K @ t_KF
    
    ### we return the object
    return t_SF

def compute_trajectory(position_init, position_goal, t_init, t_goal, t):
    desired_position = 0
    desired_velocity = 0
    
    #Total time of the trajectory
    T = t_goal - t_init
    
    #Velocity, acceleration and position constraint functions
    s_t = (10/(T**3))*(t-t_init)**3 - (15/(T**4))*(t-t_init)**4 + (6/(T**5))*(t-t_init)**5
    
    s_prime_t = (30/(T**3))*(t-t_init)**2 - (60/(T**4))*(t-t_init)**3 + (30/(T**5))*(t-t_init)**4
    
    #desired position
    desired_position = position_init + s_t*(position_goal - position_init)
    
    #desired velocity
    desired_velocity = s_prime_t*(position_goal - position_init)
    
    # we return the answer
    return desired_position, desired_velocity



## Instantiate The Robots
If you're using the real robots, put them into zero position and then run the next cell to instantiate and reset them. 

In [2]:
robot = Robot()
if use_real_robot:
    robot.reset_sensors()
else:
    robot.reset()

libdecor-gtk-WARNING: Failed to initialize GTK
Failed to load plugin 'libdecor-gtk.so': failed to init
/home/alberttsododo/.conda/envs/rob2004/lib/python3.13/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)


You can also visualize the robot to ensure that the real movement of the robot matches the 3D simulator environment (Only if you're using the real robot)

In [ ]:
import time
if use_real_robot:
    from NYUFinger.utils.vis import NYUDualFingerVisualizer
    vis = NYUDualFingerVisualizer()
    start_time = time.time()
    while time.time()-start_time < 60:
        q, dq = robot.get_state()
        robot.send_joint_torque(np.zeros(6))
        vis.show(q)
        time.sleep(0.01)
else:
    print('This visualization is meaningless for the simulated robot experiment')

## Interacting with the Robot
For this lab, you'll be using two robots at the same time. Unlike the previous labs where the $\tau$, $q$, and $dq$ were 3D vectors, in this lab robot takes 6D commands and returns 6D states. The first three numbers are for finger 1 (left) and the last three are for the finger 2 (right). Let's make sure you can read the sates and command the robot:

In [3]:
q, dq = robot.get_state()

q1 = q[:3]
q2 = q[3:]
dq1 = dq[:3]
dq2 = dq[3:]
print(f'Robot1: q={q1}, dq1={dq1}')
print(f'Robot2: q={q2}, dq1={dq2}')

Robot1: q=[-3.10451769e-01 -4.25346391e-17  2.42076164e-17], dq1=[-4.55666693e-03  1.64534085e-17 -6.13108790e-17]
Robot2: q=[-3.10451769e-01 -1.73045288e-16 -1.69097069e-19], dq1=[-4.55666693e-03 -3.00780246e-17  1.20605976e-17]


If you're using the simulator, you can also access the ground truth position of the cubes using `robot.get_cube_positions()` and reset their location using `robot.reset()`: 

In [ ]:
# Corrected code
if not use_real_robot:
    robot.reset()
    cube_pos = robot.get_cube_positions()  # Add parentheses to call the method
    print(cube_pos)
    
    cube_1 = cube_pos["cube1"]
    cube_2 = cube_pos["cube2"]
    cube_3 = cube_pos["cube3"]

    print(cube_1)
    print(cube_2)
    print(cube_3)


{'cube1': array([ 0.125     , -0.05      ,  0.02499019]), 'cube2': array([ 0.125     , -0.05      ,  0.07499019]), 'cube3': array([ 0.125     , -0.05      ,  0.12499019])}
<class 'method'>


TypeError: 'method' object is not subscriptable

## Questions
In this laboratory you are expected to re-use code from the previous laboratories (e.g. to plot, compute forward kinematics, etc) and solve the problem in both simulation and using the real robot. For both simulation and real robot deployment, you're expected to upload a short video of the robot solving the task alongside the report containing plots showing the robot reaching the desired 10cm goal for the designated cube.

1. Using an impedance controller (Lab4 - use the ``getBodyJacobianOrientedLikeSpatialFrame`` function to get the Jacobian) and one trajectory generation method seen in Lab3 write a complete controller that solves the problem: move the top two cubes away, grasp the last cube and move it up by 10cm. In the report, describe in details the controller you designed (what it does, etc) and the results you obtained.

2. Test your controller in simulation. Save and plot the results.
   
3. Test the controller on the real robots. Save and plot the results.

4. (optional - bonus) Extend the controller to rearrange the cubes - putting the bottom one on top of the two others.

In [ ]:
# we simulate for 10 seconds
run_time = 13.
dt = 0.001
num_steps = int(run_time/dt)



# the PD gains
P = np.array([3, 3, 3])
D = np.array([0.1, 0.1, 0.1])

K = np.diag(P)
D = np.diag(D)

## we allocate a few vectors to collect data while we do the simulation
# we keep track of time
time = np.zeros([num_steps])
# we will store the sensor values (angular position and velocities)
measured_positions_f1 = np.zeros([num_steps,3])
measured_velocities_f1 = np.zeros_like(measured_positions_f1)
measured_positions_f2 = np.zeros_like(measured_positions_f1)
measured_velocities_f2 = np.zeros_like(measured_positions_f1)
# we will store the end effector position (desired and measured)
measured_finger_position_f1 = np.zeros([num_steps,3])
desired_finger_position_f1 = np.zeros([num_steps,3])
measured_finger_position_f2 = np.zeros([num_steps,3])
desired_finger_position_f2 = np.zeros([num_steps,3])
t = 0

for i in range(num_steps):
    # get the current time and save it
    time[i] = dt * i
    
    time_to_goal = 3

    # we get the position and velocities of the joints
    q, dq = robot.get_state()
    q1 = q[:3]
    q2 = q[3:]
    dq1 = dq[:3]
    dq2 = dq[3:]
    
    # controller: TODO HERE IMPLEMENT YOUR CONTROLLER

    #interpolate the trajectory
    q_des_1 = [0.125, -0.05, 0.12499018]
    q_des_2 = [0.125, -0.05, 0.12499018]

    q_des_1 , dq_des_1 = compute_trajectory(q1,q_des_1, 0, time_to_goal, t)
    q_des_2 , dq_des_2 = compute_trajectory(q2,q_des_2, 0, time_to_goal, t)
    t += dt

    #Kp and Kd controller
    position_error_1 = q_des_1 - q1
    velocity_error_1 = dq_des_1 - dq1
    position_error_2 = q_des_2 - q2
    velocity_error_2 = dq_des_2 - dq2

    tau_task_1 = K @ position_error_1 + D @ velocity_error_1
    tau_task_2 = K @ position_error_2 + D @ velocity_error_2
    J1 = getBodyJacobianOrientedLikeSpatialFrame(q1)
    J2 = getBodyJacobianOrientedLikeSpatialFrame(q2)

    joint_torques1 = J1.T @ tau_task_1
    joint_torques2 = J2.T @ tau_task_2
    
    joint_torques = np.concatenate((joint_torques1, joint_torques2))
    
    # we send them to the robot and do one simulation step
    robot.send_joint_torque(joint_torques)
    sleep(dt)